In [1]:
try:
    import mesa
except ImportError:
    !pip install mesa
    import mesa

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Resource Classes

In [2]:
class Sugar(mesa.Agent):

    def __init__(self, model, pos, max_sugar):
        super().__init__(model)
        #self.grid.place_agent(sugar, pos)
        #self.pos=pos
        self.amount = max_sugar
        self.max_sugar = max_sugar

    def step(self):

        self.amount = min(self.amount + 1, self.max_sugar)
        # print("unique id :", self.unique_id, "amount: ", self.amount, "max :", self.max_sugar)



In [3]:
class Spice(mesa.Agent):

    def __init__(self, model, pos, max_spice):
        super().__init__(model)
        self.amount = max_spice
        self.max_spice = max_spice


    def step(self):

        self.amount = min(self.amount + 1, self.max_spice)
        # print("unique id :", self.unique_id, "amount: ", self.amount, "max :", self.max_spice)


In [10]:
class Trader(mesa.Agent):
    def __init__(self, model, pos, moore=False, sugar=0, spice=0, metabolism_sugar=0, metabolism_spice=0, vision=0):

        super().__init__(model)
        self.moore = moore
        self.sugar = sugar
        self.spice = spice
        self.metabolism_sugar = metabolism_sugar
        self.metabolism_spice = metabolism_spice
        self.vision = vision

    def is_occupied_by_other(self, pos):
        """
        helper function to check if the position is occupied by other traders for self.move()
        :return bool
        """
        if pos == self.pos:
            return False
        this_cell = self.model.grid.get_cell_list_contents(pos)

        for a in this_cell:
            if isinstance(a, Trader):
                return True
        else:
            return False



    def move(self):
        """
        - identify : all possible moves
        - evaluate : find best option
        - choose : to what is the best
        - move : to the best
        :return:
        """
        # 1, idenfiy all possible moves
        neighbors = [i for i in self.model.grid.get_neighborhood(self.pos, self.moore, include_center=True, radius=self.vision)
                     if not self.is_occupied_by_other(i)]

        print(self.pos, neighbors)






# model class

In [8]:
class SugarscapeG1mt(mesa.Model):
    def __init__(self, width=50, height=50, initial_population=200,
                 endowment_min=20, endowment_max=50,
                 metabolism_min=1, metabolism_max=5,
                 vision_min=1, vision_max=5):
        
        # 부모 클래스(mesa.Model) 초기화
        super().__init__()


        self.width = width
        self.height = height

        # Initialize
        self.initial_population = initial_population
        self.endowment_min = endowment_min
        self.endowment_max = endowment_max
        self.metabolism_min = metabolism_min
        self.metabolism_max = metabolism_max
        self.vision_min = vision_min
        self.vision_max = vision_max

        # .Grid property
        # torus=True로 하면 팩맨처럼 맵이 연결됩니다.
        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=True)

        self.sugar_distribution = np.genfromtxt("sugar-map.txt")
        self.spice_distribution = np.flip(self.sugar_distribution, axis=0)

        # 헬퍼 함수 make_map 호출
        # self.make_map()


        for _, (x, y) in self.grid.coord_iter():
            max_sugar = self.sugar_distribution[x, y]
            if max_sugar > 0:
                sugar = Sugar(self, (x,y), max_sugar)
                self.grid.place_agent(sugar, (x, y))
                # self.schedule.add(sugar) # Mesa 3.0+에서는 필요 없으므로 삭제

            max_spice = self.spice_distribution[x, y]
            if max_spice > 0:
                spice = Spice(self, (x,y), max_spice)
                self.grid.place_agent(spice, (x, y))
                # self.schedule.add(spice) # Mesa 3.0+에서는 필요 없으므로 삭제

        for i in range(self.initial_population):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)

            sugar = self.random.randrange(self.endowment_min, self.endowment_max + 1)
            spice = self.random.randrange(self.endowment_min, self.endowment_max + 1)

            metabolism_sugar = self.random.randrange(self.metabolism_min, self.metabolism_max + 1)
            metabolism_spice = self.random.randrange(self.metabolism_min, self.metabolism_max + 1)

            vision = self.random.randrange(self.vision_min, self.vision_max + 1)

            # Initiate individual agent
            trader = Trader(self, (x,y), moore=False, # moore는 키워드 인자로 유지
                        sugar=sugar, spice=spice, # 나머지 인자들도 키워드 인자로 명시
                        metabolism_sugar=metabolism_sugar, metabolism_spice=metabolism_spice,
                        vision=vision)
            # Place agent
            self.grid.place_agent(trader, (x,y))
            # self.schedule.add(trader) # Mesa 3.0+에서는 필요 없으므로 삭제
            # print(trader.unique_id, trader.pos, trader.sugar, trader.spice, trader.metabolism_sugar,trader.metabolism_spice, trader.vision)

    def make_map(self):

        # 도화지를 가로로 길게(12x6 크기) 준비합니다.
        plt.figure(figsize=(12, 6))

        # 첫 번째 그림: 설탕 지도 (1행 2열 중 1번째)
        plt.subplot(1, 2, 1)
        plt.imshow(self.sugar_distribution, origin="lower", cmap="Oranges")
        plt.title("Sugar Map")
        plt.colorbar()

        # 두 번째 그림: 향신료 지도 (1행 2열 중 2번째)
        plt.subplot(1, 2, 2)
        plt.imshow(self.spice_distribution, origin="lower", cmap="BuPu")
        plt.title("Spice Map")
        plt.colorbar()


    def step(self):
        # 모델의 모든 에이전트(Sugar, Spice, Trader)의 step 메서드를 호출합니다.
        # Mesa 3.0+에서는 self.agents를 통해 에이전트들을 관리합니다.
        self.agents.do("step")

    #    for sugar in self.schedule.agents_by_type[Sugar].values():
    #        sugar.step()
        # for trader
        # agent death and removal
        # Mesa 3.0+에서는 self.agents.select()를 사용하여 특정 타입의 에이전트를 가져옵니다.
        trader_shuffle = list(self.agents.select(agent_type=Trader))
        self.random.shuffle(trader_shuffle)

        for agent in trader_shuffle:
            agent.move()


    def run_model(self,step_count=1000):
        for i in range(step_count):
            self.step()


# Run Model


In [11]:
model = SugarscapeG1mt()
model.run_model(5)


(41, 26) [(40, 26), (41, 26), (41, 27), (42, 26)]
(47, 40) [(44, 40), (45, 39), (45, 40), (45, 41), (46, 38), (46, 39), (46, 40), (46, 41), (46, 42), (47, 37), (47, 38), (47, 39), (47, 40), (47, 41), (47, 43), (48, 38), (48, 39), (48, 40), (48, 41), (48, 42), (49, 39), (49, 40), (49, 41), (0, 40)]
(23, 26) [(20, 26), (21, 25), (21, 26), (21, 27), (22, 24), (22, 25), (22, 26), (22, 27), (22, 28), (23, 23), (23, 24), (23, 25), (23, 26), (23, 27), (23, 28), (23, 29), (24, 25), (24, 26), (24, 27), (24, 28), (25, 25), (25, 26), (25, 27), (26, 26)]
(45, 19) [(44, 19), (45, 18), (45, 19), (45, 20), (46, 19)]
(5, 19) [(5, 18), (5, 19), (5, 20), (6, 19)]
(47, 29) [(43, 29), (44, 28), (44, 29), (44, 30), (45, 27), (45, 28), (45, 29), (45, 30), (45, 31), (46, 26), (46, 27), (46, 28), (46, 29), (46, 30), (46, 32), (47, 25), (47, 26), (47, 27), (47, 28), (47, 29), (47, 30), (47, 31), (47, 32), (47, 33), (48, 27), (48, 28), (48, 29), (48, 30), (48, 31), (48, 32), (49, 27), (49, 28), (49, 29), (49, 3